<a href="https://colab.research.google.com/github/jeha100/google_drive/blob/main/temp/chapter_05_3_%ED%95%98%EC%A7%80%EC%9D%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블
- 앙상블 학습은 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘을 말한다.
- 랜덤 포레스트는 대표적인 결정 트리 기반의 앙상블 학습 방법이다.
- 엑스트라 트리는 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않는다.
- 그레이디언트 부스팅은 램덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법이다.


# 트리의 앙상블
- LightGBM 기억! 
  + GBM --> XGBoost --> LightGBM
  + 참고 1. 모델개발속도가 빨라졌나? 
  + 참고 2. 모델의 성능이 좋아졌나? 
- TabNet (2019)
  + 딥러닝 컨셉! 

## 랜덤 포레스트(Forest)
- 결정 트리 나무를 500개 심기
- 최종적인 결정은 투표 방식 
  + 나무-1 : 양성
  + 나무-2 : 음성
  + 나무-3 : 양성
  ..
  + 나무-500 : 양성

# 데이터 셋 분리

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, 
                                                                      target, 
                                                                      test_size=0.2, 
                                                                      random_state=42)

- p.267
  + cross_validate() 교차 검증 수행


In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 그레이디언트 부스팅
- 이전 트리의 오차를 보완하는 방식으로 사용. 
- 깊이가 얕은 트리를 사용.
- 학습률 매개변수로 속도를 조절. 
- 단점 : 속도가 느림. 

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))# 결정트리보다 속도가느림 

0.8881086892152563 0.8720430147331015


In [7]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [8]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


- 흐름
  + 0. 데이터 전처리 / 시각화
  + 1. 기본 모형으로 전체 흐름을 설계
  + 2. 여러 모형으로 비교 대조
  + 3. 교차 검증, 하이퍼파라미터 성능 비교
  + ... 